# Camera calibration using OpenCV

In [ ]:
# Configuration
calibration_dir = "camera_cal"
pattern_size = (9, 6)
example_idx = 0
show_interactively = False

In [ ]:
# load images
import os
import matplotlib.image as mplimg
import matplotlib.pyplot as plt

calibration_images = []
for image in os.listdir(calibration_dir):
    img = mplimg.imread(os.path.join(calibration_dir, image))
    calibration_images.append(img)
print("Loaded {} images.".format(len(calibration_images)))
%matplotlib inline
plt.imshow(calibration_images[example_idx])

In [ ]:
# convert to grayscale
import cv2
gray_images = []
for img in calibration_images:
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray_images.append(gray)
plt.imshow(gray_images[example_idx], cmap='gray')

In [ ]:
# Setup calibration data
import numpy as np
pattern_points = np.zeros((pattern_size[0]*pattern_size[1],3), np.float32)
pattern_points[:,:2] =np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1,2)


In [ ]:
# find corners
corners = []
object_points = []
found_indices = []
for idx, img in enumerate(gray_images):
    found, current_corners = cv2.findChessboardCorners(img, pattern_size, None)
    if found:
        corners.append(current_corners)
        object_points.append(pattern_points)
        found_indices.append(idx)
        
print("Found corners in {} of the {} images.".format(len(corners), len(gray_images)))
assert len(corners) == len(object_points)

In [ ]:
# show the corners interactively
if show_interactively:
    %matplotlib qt
    plt.ion()
    for corner_idx, img_idx in enumerate(found_indices):
        corner_img = cv2.drawChessboardCorners(calibration_images[img_idx], pattern_size, corners[corner_idx], True)
        plt.imshow(corner_img)
        plt.title("Corners {} on image {}".format(corner_idx, img_idx))
        plt.draw()
        plt.waitforbuttonpress(timeout=10)
        plt.cla()
    plt.close()

In [ ]:
# Calibrate the camera!
print("Calibrating with {} sets of object points and {} sets of corners.".format(len(object_points), len(corners)))
img_shape = gray_images[0].shape[::-1]
print("image shape:", img_shape)
#object_points.pop(7)
#object_points.pop(8)
#corners.pop(7)
#corners.pop(8)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, corners, gray_images[0].shape[::-1], None, None)
print(mtx)

In [ ]:
# Undistort example image
%matplotlib inline
undistorted_image = cv2.undistort(gray_images[example_idx], mtx, dist, None, mtx)
plt.imshow(undistorted_image, cmap='gray')

In [ ]:
if show_interactively:
    %matplotlib qt
    for image in gray_images:
        undistorted_image = cv2.undistort(image, mtx, dist, None, mtx)
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
        f.tight_layout()
        ax1.imshow(image, cmap='gray')
        ax1.set_title('Original Image')
        ax2.imshow(undistorted_image, cmap='gray')
        ax2.set_title('Undistorted Image')
        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
                
        plt.draw()
        plt.waitforbuttonpress(timeout=10)
    plt.close()

In [ ]:
import pickle

with open('camera_calibration.p', 'wb') as f:
    pickle.dump((mtx, dist), f)